In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup as BS4
from scrapy import spiders
from scrapy.spiders import CrawlSpider, Rule

import scrapy
import yaml
import getpass
import time
import json
import pandas as pd
import os


In [2]:
options = webdriver.ChromeOptions()
# options.add_argument('headless')

driver = webdriver.Chrome('./chromedriver', options=options)
driver.maximize_window()
actions = ActionChains(driver)

In [3]:
files = []
for file in os.listdir('./JSON'):
    if file.endswith('.json'):
        files.append(os.path.join('./JSON', file))

for file in files:
    with open(file) as fp:
        data = json.load(fp)

links = []
for i in data:
    links.append(i['link'])
        

In [4]:
jobs = []
for link in links:
    try:
        driver.get(link)
        
        if driver.current_url != link:
            time.sleep(3)
            driver.get(link)
            
        time.sleep(3)
        info = {}
        info['jobtitle'] = driver.find_element_by_xpath('//h1[@class="top-card-layout__title topcard__title"]').text
        info['employer'] = driver.find_element_by_xpath('//a[@data-tracking-control-name="public_jobs_topcard-org-name"]').text
        info['location'] = driver.find_element_by_xpath('//div[@class="topcard__flavor-row"]/span[2]').text
        info['seniorityLevel'] = driver.find_element_by_xpath('//ul[@class="description__job-criteria-list"]/li[1]/span').text
        info['employmentType'] = driver.find_element_by_xpath('//ul[@class="description__job-criteria-list"]/li[2]/span').text
        info['jobFunction'] = driver.find_element_by_xpath('//ul[@class="description__job-criteria-list"]/li[3]/span').text
        info['industry'] = driver.find_element_by_xpath('//ul[@class="description__job-criteria-list"]/li[4]/span').text

        try:
            info['salaryRange'] = driver.find_element_by_xpath('//div[@class="salary compensation__salary"]').text
        except:
            info['salaryRange'] = ''

        try:
            info['jobStatus'] = driver.find_element_by_xpath('//figcaption[@class="closed-job__flavor--closed"]').text
        except:
            info['jobStatus'] = 'Open'

        driver.find_element_by_xpath('//button[@data-tracking-control-name="public_jobs_show-more-html-btn"]').click()

        info['description'] = [tag.text for tag in driver.find_elements_by_xpath('//div[@class="show-more-less-html__markup"]')]
        jobs.append(info)
        
    except Exception as e:
        print(e)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h1[@class="top-card-layout__title topcard__title"]"}
  (Session info: chrome=92.0.4515.159)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h1[@class="top-card-layout__title topcard__title"]"}
  (Session info: chrome=92.0.4515.159)



KeyboardInterrupt: 

In [ ]:
jobs

In [ ]:
data = pd.DataFrame(jobs)

data.to_csv('./J/JobData.csv')